In [35]:
# to find best layer,optimizer,num of nodes

import numpy as np 
import pandas as pd

In [36]:
pr = pd.read_csv('diabetes.csv')
pr.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [37]:
# corelation of output with other parameters 
# we can remove which are not important parameters

pr.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [38]:

# in x we provide input data and in y we put output data
x = pr.iloc[:,:-1].values
y = pr.iloc[:,-1].values

In [39]:
#scaling the data (0-1)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(x)
x.shape

(768, 8)

In [40]:
# to train the data 
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.2,random_state=1)

In [41]:
#to create a model
import tensorflow
from tensorflow import keras
from tensorflow.keras import  Sequential
from tensorflow.keras.layers import  Dense,Dropout

In [42]:
# to automate the whole process of creating node and layer by hypertuning
import kerastuner as kt

In [43]:
def build_model(hp):

   model = Sequential()

   counter = 0
 
   for i in range(hp.Int('num_layers',1,10)):
     
    if counter == 0:

     model.add(Dense(hp.Int('units'+str(i),8,128,step=8),activation = hp.Choice('activation'+str(i),values=['relu','sigmoid','tanh']),input_dim = 8 ))
     #model.add(Dropout(hp.Choice('dropout'+ str(i),values =  [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
     
    else:   

     model.add(Dense(hp.Int('units'+str(i),8,128,step=8),activation =hp.Choice('activation'+str(i),values=['relu','sigmoid','tanh']),))
     #model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        
    counter +=1   

   model.add(Dense(1,activation='sigmoid'))

   model.compile(optimizer = hp.Choice('optimizer',values=['rmsprop','Adam','nadam','sgd','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])

   return model

In [44]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory = 'final',project_name= 'final1')

c:\Users\dt160\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [45]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.7804877758026123

Best val_accuracy So Far: 0.7804877758026123
Total elapsed time: 00h 00m 06s


In [46]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 40,
 'activation0': 'tanh',
 'optimizer': 'rmsprop',
 'units1': 48,
 'activation1': 'sigmoid',
 'units2': 48,
 'activation2': 'tanh',
 'units3': 80,
 'activation3': 'tanh',
 'units4': 96,
 'activation4': 'sigmoid',
 'units5': 72,
 'activation5': 'sigmoid',
 'units6': 8,
 'activation6': 'relu',
 'units7': 32,
 'activation7': 'relu',
 'units8': 120,
 'activation8': 'relu',
 'units9': 88,
 'activation9': 'tanh'}

In [49]:
model = tuner.get_best_models(num_models=1)[0]
#tuner.results_summary()

c:\Users\dt160\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\dt160\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [50]:
model.fit(x_train,y_train,epochs=200,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.7935 - loss: 0.5100 - val_accuracy: 0.7659 - val_loss: 0.5142
Epoch 8/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7588 - loss: 0.5137 - val_accuracy: 0.7659 - val_loss: 0.5058
Epoch 9/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7523 - loss: 0.4925 - val_accuracy: 0.7675 - val_loss: 0.5006
Epoch 10/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7375 - loss: 0.5022 - val_accuracy: 0.7675 - val_loss: 0.4967
Epoch 11/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7631 - loss: 0.4886 - val_accuracy: 0.7675 - val_loss: 0.4938
Epoch 12/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7714 - loss: 0.4833 - val_accuracy: 0.7659 - val_loss: 0.4919
Epoch 13/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7619 - loss: 0.4797 - val_accuracy: 0.7642 - val_loss: 0.4904
Epoch 14/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7584 - loss: 0.4800 - val_accuracy: 0.7593 - val_